In [1]:
import collections
import requests
from bs4 import BeautifulSoup
import re

# Make sure it points to the latest trt deprecated api html
url = "https://docs.nvidia.com/deeplearning/tensorrt/api/c_api/deprecated.html"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()
    # Check pattern which starts with nvinfer1::
    pattern = re.compile(r'nvinfer1::([\w:]+)')
    matches = pattern.findall(text)
    api_names = [match.split('::')[-1] for match in matches]
else:
    print("Parse Failed")

def func(file_path, api_names):
    results = collections.defaultdict(list)
    patterns = [(api, re.compile(r'\b{}\b'.format(re.escape(api)))) for api in api_names]
    
    def is_within_string(line, index):
        # Check if the character at the given index is within a string (surrounded by quotes)
        return line.count('"', 0, index) % 2 != 0 or line.count("'", 0, index) % 2 != 0
    
    with open(file_path, 'r') as file:
        for line_number, line in enumerate(file, 1):
            # Skip lines with comments
            if line.strip() == "" or line.strip().startswith("//") or line.strip().startswith("/*") or line.strip().startswith('#'):
                continue
            for api, pattern in patterns:
                match = pattern.search(line)
                if match and not is_within_string(line, match.start()):
                    results[api].append(f"Ln{line_number}")

    return results

In [2]:
ans = func("/home/yifanl/onnxruntime/onnxruntime/core/providers/tensorrt/tensorrt_execution_provider.cc", api_names) 
for k in ans:
    print(f"{k}")
    print(ans[k])

setMaxWorkspaceSize
['Ln2509', 'Ln3128']
kENABLE_TACTIC_HEURISTIC
['Ln2729', 'Ln3166']
deserializeCudaEngine
['Ln2806', 'Ln2825', 'Ln2866', 'Ln3070', 'Ln3095', 'Ln3226']
